In [ ]:
import utils.dir_utils as dir_utils
import utils.dict_utils as dict_utils
import utils.ptr_utils as ptr_utils
import utils.constants as constants 
import utils.profit as profit 
import helpers.official as official
import utils.csv_utils as csv_utils

_, input_df = dir_utils.get_data(combined=True)

In [ ]:
def setup():
    # (t[constants.TDATE], official.get_name(t), t[constants.TICKER])
    purchases = set()
    
    for _,t in input_df.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            core =  (t[constants.TDATE], official.get_name(t), t[constants.TICKER])
            
            if t[constants.TYPE] == 'Purchase' and ptr_utils.get_year(t[constants.TDATE]) < 2020: 
                purchases.add(core)

    return purchases

p = setup()


def match(p):
    # week = [name, ticker, tdate_purch, tdate_sale, profit]
    week = []
    month = []
    twomonths = []
    fourmonths = []
    eightmonths = []
    year = []
    i = 0 
    
    for (tdate_purch, name_purch, ticker_purch)  in p:
        i+=1 
        
        # 1 WEEK
        tdate_sale_week = profit.add_to_date(tdate_purch, 1)
        profit_week = profit.share_diff(ticker_purch, tdate_sale_week, tdate_purch)
        if profit_week:
            week.append([name_purch, ticker_purch, tdate_purch, tdate_sale_week, profit_week])
        
        
        # 1 MONTH
        tdate_sale_1month = profit.add_to_date(tdate_purch, 4)
        profit__1month = profit.share_diff(ticker_purch, tdate_sale_1month, tdate_purch)
        if profit__1month:
            month.append([name_purch, ticker_purch, tdate_purch, tdate_sale_1month, profit__1month])
        
        
        # 2 MONTHS
        tdate_sale_2months = profit.add_to_date(tdate_purch, 8)
        profit_2months = profit.share_diff(ticker_purch, tdate_sale_2months, tdate_purch)
        if profit_2months: 
            twomonths.append([name_purch, ticker_purch, tdate_purch, tdate_sale_2months, profit_2months])

        
        # 4 MONTHS
        tdate_sale_4months = profit.add_to_date(tdate_purch, 16)
        profit_4months = profit.share_diff(ticker_purch, tdate_sale_4months, tdate_purch)
        if profit_4months: 
            fourmonths.append([name_purch, ticker_purch, tdate_purch, tdate_sale_4months, profit_4months])

        
        # 8 MONTHS
        tdate_sale_8months = profit.add_to_date(tdate_purch, 32)
        profit_8months = profit.share_diff(ticker_purch, tdate_sale_8months, tdate_purch)
        if profit_8months:
            eightmonths.append([name_purch, ticker_purch, tdate_purch, tdate_sale_8months, profit_8months])

        
        # 12 MONTHS
        tdate_sale_year = profit.add_to_date(tdate_purch, 52)
        profit_year = profit.share_diff(ticker_purch, tdate_sale_year, tdate_purch)
        if profit_year:
            year.append([name_purch, ticker_purch, tdate_purch, tdate_sale_year, profit_year])

        print(i)

    return week, month, twomonths, fourmonths, eightmonths, year

week, month, twomonths, fourmonths, eightmonths, year = match(p)

In [ ]:
# week = [name, ticker, tdate_purch, tdate_sale, profit]
dir = dir_utils.makesubdir(constants.path_csv, constants.PROFIT)

x = [(week, "week"),
    (month, "month"),
    (twomonths, "twomonths"),
    (fourmonths, "fourmonths"),
    (eightmonths, "eightmonths"),
    (year, "year")]

print(len(week))
print(len(month))
print(len(twomonths))
print(len(fourmonths))
print(len(eightmonths))
print(len(year))

# official_name, ticker, tdate_purch : [ (tdate_sale : diff) , (tdate_sale : diff), (tdate_sale : diff), ...]
sell_profit = {}
for l, n in x:
    for (official_name, ticker, tdate_purch, tdate_sale, diff) in l:
        sell_profit = dict_utils.increment_list_in_dictionary(sell_profit, (official_name, ticker, tdate_purch), (tdate_sale, diff))

l = []
for (name, ticker, tdate_purch), v in sell_profit.items():
    r = [name, ticker, tdate_purch]
    for (x,y) in v:
        r.append(x)
        r.append(y)
    l.append(r)
    
csv_utils.make_csv_base(dir, "combinedadsatt", [constants.OFFICIAL, constants.TICKER, constants.BUYDATE, "week", constants.PROFIT, "month", constants.PROFIT, "twomonths", "profit", "fourmonths", "profit", "eightmonths", "profit", "year", "profit"], l)